## Create Synthetic Source Data 

In [0]:
%pip install dbldatagen==0.4.0.post1

In [0]:
import dbldatagen as dg

In [0]:
dbutils.widgets.text("catalog", "", "01 Catalog")
dbutils.widgets.text("schema", "", "02 Schema")
dbutils.widgets.dropdown('data_source_loc', 'delta', ['delta', 'cloudFiles'], '02 Data Source Location')

In [0]:
# https://databrickslabs.github.io/dbldatagen/public_docs/generating_cdc_data.html

def create_dataspec(row_count, partitions):
    partitions_requested = partitions
    data_rows = row_count
    spark.conf.set("spark.sql.shuffle.partitions", "auto")
    uniqueCustomers = row_count

    dataspec = (
        dg.DataGenerator(
            spark,
            rows=data_rows,
            partitions=partitions_requested,
            randomSeedMethod="hash_fieldname",
        )
        .withIdOutput()
        .withColumn("str1", "string", template=r"\\w")
        .withColumn("str2", "string", template=r"\\w")
        .withColumn(
            "event_date",
            "date",
            begin="2020-01-01",
            end="2022-12-31",
            interval="1 day",
            random=True,
        )
    )

    return dataspec

In [0]:
catalog = dbutils.widgets.get('catalog')
schema = dbutils.widgets.get('schema')

spark.sql(f'use catalog {catalog}')
spark.sql(f'create schema if not exists {schema}')

# Set partitions to 1x or 2x number of cores
dataspec = create_dataspec(row_count=1_000_000, partitions=32)
dataspec.build().createOrReplaceTempView('src_vw')

In [0]:
for i in range(8):
    i = i + 1
    spark.sql(f'create or replace table {schema}.src_table_{i:02d} as select * from src_vw')